<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [73]:
import json
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [6]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [35]:
items = list(filtered_data.items())

second_key, second_value = items[11]

text = second_value['text']
print(text)

Up to two target creatures each get +1/+0 and gain first strike until end of turn. (They deal combat damage before creatures without first strike.)
Madness {1}{R} (If you discard this card, discard it into exile. When you do, cast it for its madness cost or put it into your graveyard.)


In [64]:
class TextCleaningLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(TextCleaningLayer, self).__init__(**kwargs)

    def call(self, inputs):
        # Convert input to string if it's not already
        if not isinstance(inputs, tf.Tensor):
           inputs = tf.convert_to_tensor(inputs, dtype=tf.string)

        # 1. Lowercase the text
        cleaned_text = tf.strings.lower(inputs)

        # 2. Remove special characters (except for spaces + - /)

        cleaned_text = tf.strings.regex_replace(cleaned_text, '\n', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\.', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\{|\}', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\([^)]*\)', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[^\w\s+/\-]', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\s+', ' ')

        return cleaned_text

cleaned_text = TextCleaningLayer()
print(text)
print(cleaned_text.call(text))

Up to two target creatures each get +1/+0 and gain first strike until end of turn. (They deal combat damage before creatures without first strike.)
Madness {1}{R} (If you discard this card, discard it into exile. When you do, cast it for its madness cost or put it into your graveyard.)
tf.Tensor(b'up to two target creatures each get +1/+0 and gain first strike until end of turn madness 1 r ', shape=(), dtype=string)


In [74]:
# prompt: apply text vectorization to an input with ngram use multihot encoding

from tensorflow.keras.layers import TextVectorization

# Define the ngrams and output_mode
ngrams = 2
output_mode = "int"

vectorizer = TextVectorization(standardize=None, ngrams=ngrams, output_mode=output_mode,output_sequence_length=100)

# Adapt the vectorizer to the cleaned text
vectorizer.adapt([cleaned_text.call(text)])

# Vectorize the cleaned text
vectorized_text = vectorizer(cleaned_text.call(text))

# Print the vocabulary
print("Vocabulary:", vectorizer.get_vocabulary())
vectorized_text

embed_lyer = Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=128)
embedded_text = embed_lyer(vectorized_text)
embedded_text


Vocabulary: ['', '[UNK]', 'up to', 'up', 'until end', 'until', 'two target', 'two', 'turn madness', 'turn', 'to two', 'to', 'target creatures', 'target', 'strike until', 'strike', 'r', 'of turn', 'of', 'madness 1', 'madness', 'get +1/+0', 'get', 'gain first', 'gain', 'first strike', 'first', 'end of', 'end', 'each get', 'each', 'creatures each', 'creatures', 'and gain', 'and', '1 r', '1', '+1/+0 and', '+1/+0']


<tf.Tensor: shape=(100, 128), dtype=float32, numpy=
array([[-0.00860237,  0.04610213,  0.02143567, ...,  0.04576937,
        -0.0411425 ,  0.0271963 ],
       [ 0.03436023,  0.04108   , -0.02913995, ..., -0.00946604,
        -0.00472809,  0.01825922],
       [-0.0049292 ,  0.03479016,  0.00176198, ..., -0.00337444,
         0.02922377, -0.01973475],
       ...,
       [ 0.03339234, -0.03574512,  0.01823808, ..., -0.01043279,
         0.01446059,  0.00075433],
       [ 0.03339234, -0.03574512,  0.01823808, ..., -0.01043279,
         0.01446059,  0.00075433],
       [ 0.03339234, -0.03574512,  0.01823808, ..., -0.01043279,
         0.01446059,  0.00075433]], dtype=float32)>

In [ ]:


# Assuming 'vectorized_text' is defined from the previous code

# Define the embedding dimension
embedding_dim = 128  # You can adjust this value

# Create the embedding layer
embedding_layer = Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=embedding_dim)

# Embed the vectorized text
embedded_text = embedding_layer(vectorized_text)

# Print the shape of the embedded text
print("Shape of embedded text:", embedded_text.shape)

# Now 'embedded_text' contains the embedded word sequences.
embedded_text
